<a href="https://colab.research.google.com/github/Ganeshpy3/chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
import keras 
from keras.layers import Dense
from keras.models import Sequential
import pandas as pd
import numpy as np
import json
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
import time
import random

In [97]:
file=open("/content/sample_data/intent.json")

In [98]:
data=json.load(file)

In [99]:
print(data["intends"][0])

{'tags': 'welcome', 'patterns': ['hi', 'hello', 'hey', 'hi', 'hey there'], 'response': ['Hello', 'Hey whats up', 'Hi']}


In [100]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [101]:
intenddict={}
for intend in data["intends"]:
  intenddict[intend["tags"]]=[i for i in intend["response"]]

In [102]:
ps=PorterStemmer()

In [103]:
allwords=[]
intent=[]
text=[]
for intend in data["intends"]:
  # print(intend["tags"])
  inte=intend["tags"]
  for p in intend["patterns"]:
    intent.append(inte)
    li=[]
    p=word_tokenize(p)
    for val in p:
       valu=ps.stem(val.lower())
       li.append(valu)
       allwords.append(valu)
    text.append(" ".join(li))

In [107]:
opshape=len(set(intent))

In [108]:
df=pd.DataFrame({"Text":text,"Intent":intent})

In [109]:
le=LabelEncoder()

In [110]:
Y=le.fit_transform(df["Intent"])

In [111]:
Y

array([10, 10, 10, 10, 10,  9,  9,  9,  2,  2,  2,  7,  7,  7,  1,  1,  1,
        4,  4,  4,  6,  6,  8,  8,  8,  0,  0,  0,  5,  5,  5,  5,  3,  3])

In [112]:
allwords=set(allwords)

In [113]:
def bag_of_words(words,allwords=allwords):
  words=word_tokenize(words)

  data=np.zeros(len(allwords))
  for idx,value in enumerate(allwords):
    if value in words:
      data[idx]=1

  return data




In [114]:
Xbag=[]
for sentence in df["Text"]:
  Xbag.append(bag_of_words(sentence,allwords))



In [115]:
X=pd.DataFrame(Xbag,columns=allwords)

In [116]:
X.shape

(34, 45)

In [118]:
model=Sequential()
model.add(Dense(units=1000,activation="relu",kernel_initializer='he_uniform',input_dim=X.shape[1]))
model.add(Dense(units=50,activation="relu",kernel_initializer="he_uniform"))
model.add(Dense(units=opshape,activation="softmax"))

In [119]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 1000)              46000     
_________________________________________________________________
dense_10 (Dense)             (None, 50)                50050     
_________________________________________________________________
dense_11 (Dense)             (None, 11)                561       
Total params: 96,611
Trainable params: 96,611
Non-trainable params: 0
_________________________________________________________________


In [120]:
model.compile(loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [121]:
classifier=model.fit(X,Y,epochs=20)

Epoch 1/20
2/2 [==============================] - 1s 8ms/step - loss: 2.4203 - accuracy: 0.0600
Epoch 2/20
2/2 [==============================] - 0s 4ms/step - loss: 1.9682 - accuracy: 0.4700
Epoch 3/20
2/2 [==============================] - 0s 8ms/step - loss: 1.6788 - accuracy: 0.7598
Epoch 4/20
2/2 [==============================] - 0s 7ms/step - loss: 1.4480 - accuracy: 0.7206
Epoch 5/20
2/2 [==============================] - 0s 6ms/step - loss: 1.2654 - accuracy: 0.8707
Epoch 6/20
2/2 [==============================] - 0s 6ms/step - loss: 1.1380 - accuracy: 1.0000
Epoch 7/20
2/2 [==============================] - 0s 5ms/step - loss: 0.9996 - accuracy: 0.9700
Epoch 8/20
2/2 [==============================] - 0s 7ms/step - loss: 0.8908 - accuracy: 1.0000
Epoch 9/20
2/2 [==============================] - 0s 4ms/step - loss: 0.7864 - accuracy: 1.0000
Epoch 10/20
2/2 [==============================] - 0s 5ms/step - loss: 0.6885 - accuracy: 0.9700
Epoch 11/20
2/2 [======================

In [122]:
a=bag_of_words("who are you",allwords)

In [123]:
b=pd.DataFrame([a])

In [124]:
np.argmax(model.predict(b))

2

In [135]:
def predict(inp):
  if len(model.predict(inp)[model.predict(inp)>0.5])>0:
    return le.inverse_transform([np.argmax(model.predict(inp))])
  else:
    return "Chitti: Sorry can't understand"

In [136]:
def command(data):
  p=word_tokenize(data)
  li=[]
  for valu2 in p:
    value=ps.stem(valu2.lower())
    li.append(value)
  inp=bag_of_words(" ".join(li))
  output=predict(pd.DataFrame([inp]))
  return output

In [139]:
def chat():
  while True:
    inp=input("You:")
    if inp=="exit":
      return "Chat Ended"
    else:
      if command(inp)[0] in intenddict:
        print("Chitti: ",random.choice(intenddict[command(inp)[0]]))
      else:
        print(command(inp))

In [140]:

chat()

You:hey
Chitti:  Hello
You:how are you
Chitti:  :)fine,How are you ?
You:wats up
Chitti: Sorry can't understand
You:wat are you doing
Chitti: Sorry can't understand
You:who r you
Chitti: Sorry can't understand
You:what r you
Chitti: Sorry can't understand
You:what are you
Chitti:  Iam Chitti a chatbot
You:wat r u
Chitti: Sorry can't understand
You:wat are you
Chitti: Sorry can't understand
You:hey
Chitti:  Hey whats up
You:what r you
Chitti: Sorry can't understand
You:wat are you
Chitti: Sorry can't understand
You:how r you
Chitti:  :)fine,How are you ?
You:tell me joke
Chitti:  how did barber wins the race 


 he knew the shortcuts
You:nice
Chitti:  :) <3
You:bye
Chitti:  see you :)
You:exit


'Chat Ended'